In [4]:
# Load required libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import GaussianNB

from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, make_scorer, recall_score, precision_score

In [5]:
adult_income_df = pd.read_csv('cleaned.csv')

In [6]:
adult_income_df.head(10)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,39,State-gov,11.258240,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,11.330336,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,12.281393,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,12.366153,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,12.732011,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,non_usa,<=50K
5,37,Private,12.558777,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,11.984097,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,non_usa,<=50K
7,52,Self-emp-not-inc,12.253157,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,10.731624,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,11.979479,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [7]:
adult_income_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30718 entries, 0 to 30717
Data columns (total 15 columns):
age               30718 non-null int64
workclass         30718 non-null object
fnlwgt            30718 non-null float64
education         30718 non-null object
education-num     30718 non-null int64
marital-status    30718 non-null object
occupation        30718 non-null object
relationship      30718 non-null object
race              30718 non-null object
sex               30718 non-null object
capital-gain      30718 non-null int64
capital-loss      30718 non-null int64
hours-per-week    30718 non-null int64
native-country    30718 non-null object
target            30718 non-null object
dtypes: float64(1), int64(5), object(9)
memory usage: 3.5+ MB


In [8]:
# encode categorical variables using Label Encoder
# select all categorical variables
df_categorical = adult_income_df.select_dtypes(include=['object'])
df_categorical.head()

# apply Label encoder to df_categorical
le = preprocessing.LabelEncoder()
df_categorical = df_categorical.apply(le.fit_transform)
df_categorical.head()

# concat df_categorical with original df
adult_income_df = adult_income_df.drop(df_categorical.columns, axis=1)
adult_income_df = pd.concat([adult_income_df, df_categorical], axis=1)
adult_income_df.head(10)

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass,education,marital-status,occupation,relationship,race,sex,native-country,target
0,39,11.258240,13,2174,0,40,5,9,4,0,1,4,1,0,0
1,50,11.330336,13,0,0,13,4,9,2,3,0,4,1,0,0
2,38,12.281393,9,0,0,40,2,11,0,5,1,4,1,0,0
3,53,12.366153,7,0,0,40,2,1,2,5,0,2,1,0,0
4,28,12.732011,13,0,0,40,2,9,2,9,5,2,0,1,0
5,37,12.558777,14,0,0,40,2,12,2,3,5,4,0,0,0
6,49,11.984097,5,0,0,16,2,6,3,7,1,2,0,1,0
7,52,12.253157,9,0,0,45,4,11,2,3,0,4,1,0,1
8,31,10.731624,14,14084,0,50,2,12,4,9,1,4,0,0,1
9,42,11.979479,13,5178,0,40,2,9,2,3,0,4,1,0,1


In [9]:
from sklearn.preprocessing import StandardScaler
numerical_column_names = ['age','fnlwgt','education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
scaler = StandardScaler()

adult_income_df[numerical_column_names] = scaler.fit_transform(adult_income_df[numerical_column_names])

C:\Users\keshav\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\keshav\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [10]:
adult_income_df.head(10)

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass,education,marital-status,occupation,relationship,race,sex,native-country,target
0,0.042416,-1.155008,1.119909,0.142438,-0.219179,-0.079207,5,9,4,0,1,4,1,0,0
1,0.880958,-1.040404,1.119909,-0.147516,-0.219179,-2.331988,4,9,2,3,0,4,1,0,0
2,-0.033815,0.471392,-0.441111,-0.147516,-0.219179,-0.079207,2,11,0,5,1,4,1,0,0
3,1.109651,0.606125,-1.221621,-0.147516,-0.219179,-0.079207,2,1,2,5,0,2,1,0,0
4,-0.796125,1.187691,1.119909,-0.147516,-0.219179,-0.079207,2,9,2,9,5,2,0,1,0
5,-0.110046,0.912319,1.510164,-0.147516,-0.219179,-0.079207,2,12,2,3,5,4,0,0,0
6,0.804727,-0.001189,-2.002131,-0.147516,-0.219179,-2.081679,2,6,3,7,1,2,0,1,0
7,1.033420,0.426507,-0.441111,-0.147516,-0.219179,0.337974,4,11,2,3,0,4,1,0,1
8,-0.567432,-1.992113,1.510164,1.730916,-0.219179,0.755156,2,12,4,9,1,4,0,0,1
9,0.271109,-0.008529,1.119909,0.543092,-0.219179,-0.079207,2,9,2,3,0,4,1,0,1


In [11]:
# convert target variable income to categorical
adult_income_df['target'] = adult_income_df['target'].astype('category')

X = adult_income_df.drop('target', axis=1)
y = adult_income_df['target'] 

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=43)

# GaussianNB

In [13]:
clf = GaussianNB()
clf.fit(X_train, y_train)

# Making predictions
y_pred_default = clf.predict(X_test)

In [14]:
# Printing classification report
print(classification_report(y_test, y_pred_default))

local_confusion_matrix = metrics.confusion_matrix(y_test, y_pred_default )

# Printing confusion matrix and accuracy
print('Confusion Matrix:')
print(local_confusion_matrix)

# Accuracy, precision, recall and f1 score
print('\nScores:')
accuracy = metrics.accuracy_score(y_test, y_pred_default)
precision = metrics.precision_score(y_test, y_pred_default)
recall = metrics.recall_score(y_test, y_pred_default)
f1_score = metrics.f1_score(y_test, y_pred_default)
roc_auc = metrics.roc_auc_score(y_test, y_pred_default)
sensitivity = recall
specificity =  local_confusion_matrix[0,0]/(local_confusion_matrix[0,0]+local_confusion_matrix[0,1])
print(" Accuracy {0:.3f}, \n Sensitivity {1:.3f}, \n Specificity {2:.3f}, \n Precision {3:.3f}, \n Recall {4:.3f}, \n f1_score {5:.3f}, \n roc_auc {6:.3f}".format(
    accuracy, sensitivity,specificity, precision,recall,f1_score,roc_auc))

              precision    recall  f1-score   support

           0       0.82      0.95      0.88      6972
           1       0.69      0.34      0.46      2244

   micro avg       0.80      0.80      0.80      9216
   macro avg       0.75      0.65      0.67      9216
weighted avg       0.79      0.80      0.78      9216

Confusion Matrix:
[[6620  352]
 [1477  767]]

Scores:
 Accuracy 0.802, 
 Sensitivity 0.342, 
 Specificity 0.950, 
 Precision 0.685, 
 Recall 0.342, 
 f1_score 0.456, 
 roc_auc 0.646


In [45]:
y_pred_proba_default = clf.predict_proba(X_test)
gnb_df = pd.DataFrame({'row_id':y_test.index , 'Actual':y_test.values,'gnb_proba':y_pred_proba_default[:,1]})
gnb_df.to_csv('gnb_df.csv', index=False)

In [44]:
gnb_df.head(10)

,row_id,Actual,gnb_proba
0,20810,0,0.000717
1,2860,0,0.071479
2,4408,0,0.001196
3,25747,0,0.013395
4,23441,0,0.005260
5,1825,0,0.011017
6,8438,0,0.001910
7,16783,0,0.000630
8,145,0,0.000007
9,380,1,0.533800
